# A3 — Detecção de Evento de Escorregamento (Médio)

Detecte eventos de escorregamento a partir de trechos tácteis + corrente de motor (classificação binária). Otimize AUROC.

### Contexto
**Escorregamento (slip)** ocorre quando um gripper ou pé perde tração: forças tangenciais excedem a fricção e a área de contato cisalha. Matrizes tácteis (canais 0–5) capturam mudanças de pressão enquanto a corrente do motor (canais 6–7) reflete esforço extra para manter o movimento. Detectar cedo permite ajustar força de preensão, reprojetar colocação do pé ou modular a passada antes da falha.

Geramos trechos via [`common.data_utils.slip_detection_dataset`](common/data_utils.py): negativos exibem ruído nominal; positivos injetam pulsos tácteis locais e rampas ascendentes de corrente para emular slip incipiente.

### Por que métodos quânticos aqui?
Os indícios de escorregamento são sutis em janelas ruidosas. Comprimir estatísticas (energia, bandas espectrais, inclinações) em um embedding de baixa dimensão e testar kernels quânticos ou classificadores variacionais pode enriquecer fronteiras de decisão sob orçamento reduzido de qubits.

## Especificação de Entrada e Saída
**Entrada:** `X_train (N×W×8)`, `y_train (N,)`, `X_test (M×W×8)` com $W=64$ nos dados fornecidos  
**Saída:** `scores (M,)` escores reais (maior ⇒ mais provável escorregamento)  
**Tipo de retorno:** array float (ex.: `np.float64`)

Orientações de runtime/recursos: qubits ≤ 10, passos do otimizador ≤ 150; mantenha extração de características vetorizada.

### Dicas
- Canais tácteis 0–2 têm padrões de pulsos; considere potência de banda, wavelets ou energia em janela curta.
- Canal de corrente 7 cresce durante slip; inclinação (slope) ou soma acumulada são informativos.
- Correlacione pulsos tácteis com a inclinação da corrente para reduzir alarmes falsos.
- Ideia quântica: mapear estatísticas extraídas via [`common.quantum_utils.feature_map`](common/quantum_utils.py) e treinar um QSVM/VQC para maximizar AUROC.

_Orientação de baseline:_ [`common.baselines.baseline_slip`](common/baselines.py) tipicamente atinge AUROC ≈ 0.83–0.86 em seeds variadas, deixando margem acima do limiar público 0.82.

## Setup

In [ ]:
import sys
import os
# Adiciona o diretório pai (qhacka-2025) ao caminho do Python
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import numpy as np
from common import data_utils as du
from common import baselines as bl  
from common import quantum_utils as qu
np.random.seed(1337)

## Baseline (referência)

In [2]:
Xtr, ytr = du.slip_detection_dataset(n=500, w=64, seed=0, pos_rate=0.3)
Xte, yte = du.slip_detection_dataset(n=200, w=64, seed=1, pos_rate=0.3)
scores = bl.baseline_slip(Xtr, ytr, Xte)
from sklearn.metrics import roc_auc_score
au = roc_auc_score(yte, scores)
print(f"AUROC do baseline: {au:.3f}")


AUROC do baseline: 1.000


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(Xtr, ytr, Xte):
    
    # --- Início da Solução VQC para Classificação ---

    # 1. Importar bibliotecas
    import pennylane as qml
    from pennylane import numpy as pnp
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from common import quantum_utils as qu

    # 2. Engenharia de Features (Clássica)
    # Xtr tem shape (N, 64, 8)
    def extract_features(X):
        # Média da potência FFT dos 3 primeiros canais (táteis)
        # np.abs(np.fft.rfft(...)).mean(axis=1)
        f = pnp.mean(pnp.abs(pnp.fft.rfft(X[:, :, 0:3], axis=1)), axis=1)
        
        # 'Slope' (inclinação) do canal 7 (corrente)
        slope = (X[:, -1, 7] - X[:, 0, 7]).reshape(-1, 1)
        
        # Concatena em um vetor de 4 features (3 de FFT + 1 de slope)
        return pnp.concatenate([f, slope], axis=1)

    Xtr_feat = extract_features(Xtr)
    Xte_feat = extract_features(Xte)

    # 3. Pré-processamento (Scaler)
    n_qubits = 4 
    
    scaler = StandardScaler()
    Xtr_norm = scaler.fit_transform(Xtr_feat)
    Xte_norm = scaler.transform(Xte_feat)

    # 4. Normalizar Alvos Y
    # O VQC retorna [-1, 1]. Nossos alvos 'y' (ytr) são 0 ou 1.
    # Vamos mapear 0 -> -1 e 1 -> +1.
    ytr_norm = (ytr * 2) - 1

    # 5. Configurar o VQC (usando o utilitário do hackathon)
    n_layers = 2
    # `make_classifier` nos dá um VQC que retorna 1 score
    circuit, weight_shapes, dev = qu.make_classifier(n_qubits=n_qubits, n_layers=n_layers, shots=None)

    # 6. Definir a Função de Custo (Erro Quadrático Médio)
    # O primeiro argumento (argnum=0) são os 'weights'
    def cost(weights, x_sample, y_target_sample):
        # O circuito retorna um score em [-1, 1]
        pred = circuit(x_sample, weights) 
        # Comparamos com o y normalizado (-1 ou 1)
        loss = (pred - y_target_sample)**2 
        return loss

    # 7. Treinamento (com Gradiente Manual)
    n_steps = 150 #
    
    weights = pnp.random.random(size=weight_shapes["weights"], requires_grad=True)
    opt = qml.AdamOptimizer(stepsize=0.01)
    
    # Criamos a função de gradiente para o argumento 0 (weights)
    grad_fn = qml.grad(cost, argnum=0)

    for i in range(n_steps):
        idx = pnp.random.randint(0, len(Xtr_norm))
        x_sample = Xtr_norm[idx]
        y_target_sample = ytr_norm[idx]
        
        # Forçamos o Autograd a não treinar os dados
        x_sample = pnp.array(x_sample, requires_grad=False)
        y_target_sample = pnp.array(y_target_sample, requires_grad=False)
        
        grads = grad_fn(weights, x_sample, y_target_sample)
        weights = opt.apply_grad(grads, weights)

    # 8. Previsão
    preds_scores = []
    for x_sample in Xte_norm:
        # O 'score' é a saída bruta do circuito
        score = circuit(x_sample, weights)
        preds_scores.append(score)
        
    # A tarefa pede os scores, não as classes
    return pnp.array(preds_scores).numpy()

    # --- Fim da Solução VQC ---

## Testes públicos (rápidos)

In [4]:
from sklearn.metrics import roc_auc_score
Xtr, ytr = du.slip_detection_dataset(n=400, w=64, seed=10, pos_rate=0.3)
Xte, yte = du.slip_detection_dataset(n=200, w=64, seed=11, pos_rate=0.3)
scores = solve(Xtr, ytr, Xte)
au = roc_auc_score(yte, scores)
print("AUROC público:", au)
assert scores.shape == (len(yte),)
assert au >= 0.82, "AUROC público abaixo do limiar"
print("OK")


AUROC público: 1.0
OK


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.

In [5]:
# --- CÉLULA DE VALIDAÇÃO (seed=99) ---
print("--- Iniciando Teste de Validação (Robustedez) ---")
from sklearn.metrics import roc_auc_score

# Usar um seed diferente (ex: 99) para simular os testes ocultos da banca
Xtr_val, ytr_val = du.slip_detection_dataset(n=400, w=64, seed=99, pos_rate=0.3)
Xte_val, yte_val = du.slip_detection_dataset(n=200, w=64, seed=100, pos_rate=0.3)

# Rodar a mesma solução 'solve'
scores_val = solve(Xtr_val, ytr_val, Xte_val)
au_val = roc_auc_score(yte_val, scores_val)

print("AUROC Validação (seed=99):", au_val)

# Pegar o AUROC do teste público (variável 'au' da célula anterior)
try:
    print("AUROC Público   (seed=10):", au) 
    print(f"Diferença: {abs(au - au_val):.4f}")
    
    # Veredito: 1. A validação tem que passar na meta. 2. A diferença tem que ser pequena.
    assert au_val >= 0.82, "AUROC de validação falhou no limiar!"
    assert abs(au - au_val) < 0.10, "Overfitting detectado! A diferença é muito alta."
    print("VEREDITO: Solução robusta, não houve overfitting.")
except NameError:
    print("Execute a célula de Teste Público primeiro para comparar.")

print("--- Fim do Teste de Validação ---")

--- Iniciando Teste de Validação (Robustedez) ---
AUROC Validação (seed=99): 1.0
AUROC Público   (seed=10): 1.0
Diferença: 0.0000
VEREDITO: Solução robusta, não houve overfitting.
--- Fim do Teste de Validação ---
